In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
xleft = 0
xright = 2
xmiddle = 1
h = (xright-xleft)/700
T = 0.1
tau = T/700
gamma = 1.4
N = int((xright-xleft)/h)
q = 0.05

In [3]:
f0 = np.zeros((3, N+1))
f_middle = np.zeros((3, N+1))
f1 = np.zeros((3, N+1))
f1_smooth = np.zeros((3, N+1))
Q_minus = np.zeros((3))
Q_plus = np.zeros((3))

In [4]:
def F(vector, param, coordn):
    if param == 0:
        return vector[1, coordn]
    if param == 1:
        return vector[1,coordn]**2/vector[0,coordn] + (vector[2,coordn]-vector[1,coordn]**2/(2*vector[0,coordn]))*(gamma-1)
    if param == 2:
        return (vector[1,coordn]/vector[0,coordn])*(vector[2,coordn]+(vector[2,coordn]-vector[1,coordn]**2/(2*vector[0,coordn]))*(gamma-1))

In [5]:
p1 = 0.7
p2 = 0.8
ro1 = 0.9
ro2 = 1.1
u1 = 1
u2 = 0.6

In [6]:
for i in range(0, int((xmiddle-xleft)/h)+1):
    f0[0,i] = ro1
    f0[1,i] = ro1*u1
    f0[2,i] = p1/(gamma-1) + (ro1*u1)**2/(2*ro1)
for i in range(int((xmiddle-xleft)/h)+1, N+1):
    f0[0,i] = ro2
    f0[1,i] = ro2*u2
    f0[2,i] = p2/(gamma-1) + (ro2*u2)**2/(2*ro2)

In [7]:
def edge_condition(param, side):
    if side == 0:
        if param == 0:
            return ro1
        if param == 1:
            return ro1*u1
        if param == 2:
            return p1/(gamma-1) + (ro1*u1)**2/(2*ro1)
    if side == 1:
        if param == 0:
            return ro2
        if param == 1:
            return ro2*u2
        if param == 2:
            return p2/(gamma-1) + (ro2*u2)**2/(2*ro2)   

In [13]:
for n in range(0, int(T/tau)):
    for side in range(0,2):
        for param in range(0,3):
            f0[param, side*N] = edge_condition(param, side)
            f_middle[param, side*N] = edge_condition(param, side)
            f1[param, side*N] = edge_condition(param, side)
            f1_smooth[param, side*N] = edge_condition(param, side)
    for i in range(1, N):
        for param in range(0,3):
            f_middle[param, i] = f0[param, i]-(tau/h)*(F(f0, param, i+1) - F(f0,param,i))
        for param in range(0,3):
            f1[param, i] = (f0[param, i] + f_middle[param, i])/2 - (tau/(2*h))*(F(f_middle,param,i) - F(f_middle,param,i-1))
    for j in range(1, N):
        if (j-2)<0:
            j_m2 = 0
        else:
            j_m2 = j-2
        if (j+2)>N:
            j_p2 = N
        else:
            j_p2 = j+2            
        
        Dmm = f1[0,j-1] - f1[0,j_m2]
        Dm = f1[0,j] - f1[0,j-1]
        Dp = f1[0,j+1] - f1[0,j]
        Dpp = f1[0,j_p2] - f1[0,j+1]
        if (Dmm*Dm <= 0) or (Dm*Dp <= 0):
            for param in range(0,3):
                Q_minus[param]  = f1[param,j] - f1[param, j-1]
        else:
            for param in range(0,3):
                Q_minus[param]  = 0           
        if (Dm*Dp <= 0) or (Dp*Dpp <= 0):
            for param in range(0,3):
                Q_plus[param]  = f1[param,j+1] - f1[param, j]                
        else:
            for param in range(0,3):
                Q_plus[param]  = 0       
                
        for param in range(0,3):                
            f1_smooth[param,j] = f1[param,j] + q*(Q_plus[param]-Q_minus[param])
            
    f0=np.copy(f1_smooth)
    #f0=np.copy(f1)

In [14]:
x = np.zeros((N+1))    
for i in range(0, N+1):
    x[i] = i*h
density = np.zeros((N+1))
for i in range(0, N+1):
    density[i] = f0[0,i]

pressure = np.zeros((N+1))
for i in range(0, N+1):
    pressure[i] = (f0[2,i]-f0[1,i]**2/(2*f0[0,i]))*(gamma-1)
    #pressure[i] = F(f0, 1,i) - f0[1,i]**2/f0[0,i]
    
velocity= np.zeros((N+1))
for i in range(0, N+1):
    velocity[i] = f0[1,i]/f0[0,i]
    
E= np.zeros((N+1))
for i in range(0, N+1):
    E[i] = f0[2,i]